In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import os

sns.set(style='ticks')
from MATSQdataPros import *

<div style="text-align: center;">
    <img src="model.png" alt="model" width="550"/>
</div>

In [2]:
! ls

 ATB					      'Other Chains'
 FF_parms.png				       parametrization.py
 fixed_data_MATSQ.lammps		       parametrize_nafion_12reps.ipynb
'generate LAMMPS files for C23HF47O5S.ipynb'   polymer_12reps.mol2
 IONOMER_12reps_MODIFIED_DREIDING_.mol	       polymer_12reps.pdb
 IONOMER3_.mol				       polymer_12_reps_weird_naming.pdb
 ionomer_parms.lammps			       polymer_3reps.mol
 MATSQdataPros.py			       __pycache__
 model.png				       REPS
 NAFION_3repsPOLYMER.data


In [3]:
file_ = "fixed_data_MATSQ.lammps"

In [4]:
structure = get_structure(get_file(file_), [88,923])
structure["charge"] = [0.0]*len(structure)

bonds = get_bonds(get_file(file_),[1766,2601])
angles = get_angles(get_file(file_),[2604,4236])
dihedrals = get_dihedrals(get_file(file_),[4239,-1])

#### Voy a cambiar las cargas conforme el bonding de las particulas, ya que no hay orden en el nombramiento del main chain.

In [5]:
F = structure[structure["dreiding elemts"].isin(["F_"])]["atom id"]

In [6]:
S   = structure[structure["dreiding elemts"].isin(["S_3"])]["atom id"]
H   = structure[structure["dreiding elemts"].isin(["H"])]["atom id"]
O_2 = structure[structure["dreiding elemts"].isin(["O_2"])]["atom id"]
O_3 = structure[structure["dreiding elemts"].isin(["O_3"])]["atom id"]
O_H = bonds[bonds["atom2"].isin(H.values)]["atom1"].values # O bonded to H and S (single bond)

# Not so simple | side chain
C_3216 = bonds[bonds["atom2"].isin(S.values)]
C_3216 = C_3216[~C_3216["atom1"].isin(list(O_2.values) + list(O_3.values))]["atom1"].values

F_3278 = bonds[bonds["atom2"].isin(C_3216)]
C_3218 = F_3278[~F_3278["atom1"].isin(F.values)]["atom1"].values
F_3278 = F_3278[F_3278["atom1"].isin(F.values)]["atom1"].values

O_2604 = bonds[bonds["atom1"].isin(C_3218)]
O_2604 = O_2604[O_2604["atom2"].isin(O_3)]["atom2"].values
F_1662 = bonds[bonds["atom2"].isin(C_3218)]
F_1662 = F_1662[F_1662["atom1"].isin(F)]["atom1"].values

C_4010 = bonds[bonds["atom2"].isin(O_2604)]
C_4010 = C_4010[~C_4010["atom1"].isin(C_3218)]["atom1"].values

F_1913 = bonds[bonds["atom2"].isin(C_4010)]
F_1913 = F_1913[F_1913["atom1"].isin(F)]["atom1"].values
C_4947 = bonds[bonds["atom1"].isin(C_4010)]
C_4947 = C_4947[~C_4947["atom2"].isin(O_3)]["atom2"].values
C_3228 = bonds[bonds["atom2"].isin(C_4010)]
C_3228 = C_3228[~C_3228["atom1"].isin(F_1913)]["atom1"].values

F_1649 = bonds[bonds["atom2"].isin(C_4947)]
F_1649 = F_1649[F_1649["atom1"].isin(F)]["atom1"].values

F_1637 = bonds[bonds["atom2"].isin(C_3228)]
F_1637 = F_1637[F_1637["atom1"].isin(F)]["atom1"].values

# Main chain

F_sidechain = list(F_3278) + list(F_1662) + list(F_1913) + list(F_1649) + list(F_1637)
F_mainchain = structure[structure["atom id"].isin(F)]
F_mainchain = F_mainchain[~F_mainchain["atom id"].isin(F_sidechain)]["atom id"].values

C_ends = [499,732] # [close to rep-7, close to side chain] 

In [7]:
# Fix charges
structure.loc[S.index, "charge"]   = 1.4124
structure.loc[O_2.index, "charge"] = -0.6320
structure.loc[structure[structure["atom id"].isin(O_H)].index, "charge"]    = -0.6320
structure.loc[structure[structure["atom id"].isin(C_3216)].index, "charge"] =  0.3216
structure.loc[structure[structure["atom id"].isin(C_3218)].index, "charge"] =  0.3218
structure.loc[structure[structure["atom id"].isin(F_3278)].index, "charge"] = -0.3278
structure.loc[structure[structure["atom id"].isin(O_2604)].index, "charge"] = -0.2604
structure.loc[structure[structure["atom id"].isin(F_1662)].index, "charge"] = -0.1662
structure.loc[structure[structure["atom id"].isin(C_4010)].index, "charge"] =  0.4010
structure.loc[structure[structure["atom id"].isin(F_1913)].index, "charge"] = -0.1913
structure.loc[structure[structure["atom id"].isin(C_4947)].index, "charge"] =  0.4947
structure.loc[structure[structure["atom id"].isin(C_3228)].index, "charge"] =  0.3228
structure.loc[structure[structure["atom id"].isin(F_1649)].index, "charge"] = -0.1649
structure.loc[structure[structure["atom id"].isin(F_1637)].index, "charge"] = -0.1637
structure.loc[structure[structure["atom id"].isin(C_ends)].index, "charge"] =  0.5769

# There is only one F that doesn't have this value,  to be fixed later
structure.loc[structure[structure["atom id"].isin(F_mainchain)].index, "charge"] = -0.1923
structure.loc[structure[structure["atom id"].isin(C_ends)].index, "charge"] = 0.5769

# WARNING: only run once
O_2742 = structure[(structure["dreiding elemts"].isin(["O_3"])) & (structure["charge"]==0.0)]["atom id"].values
structure.loc[structure[structure["atom id"].isin(O_2742)].index, "charge"] = -0.2742

C_3218 = bonds[bonds["atom2"].isin(O_2742)]
C_3218 = C_3218[~C_3218["atom1"].isin(C_3228)]["atom1"].values
structure.loc[structure[structure["atom id"].isin(C_3218)].index, "charge"] = 0.3218

F_1641 = bonds[bonds["atom2"].isin(C_3218)]
F_1641 = F_1641[F_1641["atom1"].isin(F)]["atom1"].values
structure.loc[structure[structure["atom id"].isin(F_1641)].index, "charge"] = -0.1641

C_mainchain = structure[
    (structure["charge"] == 0.0) & (structure["dreiding elemts"].isin(["C_3"]))
]["atom id"].values
structure.loc[structure[structure["atom id"].isin(C_mainchain)].index, "charge"] =  0.3846

In [8]:
structure["charge"].sum()

-11.999999999999986

## FIX THE STRUCTURE (remove H)

In [9]:
len(structure) - 12

824

In [10]:
structure = structure[~structure["atom id"].isin(H)]
len(structure)

824

In [11]:
len(bonds) - 12

823

In [12]:
bonds = bonds[~bonds["atom2"].isin(H)]
bonds = bonds.reset_index(drop=True)
bonds.loc[bonds.index, "bond id"] = list(range(1,len(bonds)+1))
len(bonds)

823

In [13]:
len(angles) - 12

1620

In [14]:
angles = angles[~angles["atom1"].isin(H)]
angles = angles.reset_index(drop=True)
angles.loc[angles.index, "angle id"] = list(range(1,len(angles)+1))
len(angles)

1620

In [15]:
len(dihedrals) - 3*12

2312

In [16]:
dihedrals = dihedrals[~dihedrals["atom1"].isin(H)]
dihedrals = dihedrals.reset_index(drop=True)
dihedrals.loc[dihedrals.index, "dihedral id"] = list(range(1,len(dihedrals)+1))
len(dihedrals)

2312

## FIX FF PARAMETERS

[DREIDING Force Field](https://pubs.acs.org/doi/abs/10.1021/j100389a010)


<div style="text-align: center;">
    <img src="FF_parms.png" alt="FF_parms" width="800"/>
</div>

In [17]:
CB = list(C_ends) + list(C_mainchain) + list(C_3218)
C  = structure[structure["dreiding elemts"].isin(["C_3"])]
C  = list(C[~C["atom id"].isin(CB)]["atom id"].values)
O  = list(structure[structure["dreiding elemts"].isin(["O_3", "O_2"])]["atom id"].values)
S  = list(S.values)
F  = list(F.values)

In [18]:
MODEL_structure = structure[["atom id","charge","x","y","z"]].reset_index(drop=True).copy()

In [19]:
dct_types = dict(zip( CB + C + F + O + S , [1]*len(CB) + [2]*len(C) + [3]*len(F) + [4]*len(O) + [5]*len(S) ))

In [20]:
MODEL_structure["type"] = MODEL_structure["atom id"].map(dct_types)

In [21]:
MODEL_bonds = bonds[["bond id","atom1","atom2"]].copy()
MODEL_bonds["X-X"] = (
    MODEL_bonds["atom1"].map(dct_types).astype(str) + "-" + 
    MODEL_bonds["atom2"].map(dct_types).astype(str)
)

In [22]:
"""
'In DREIDING we set all energy parameters for single bonds to K=700 (kcal/mol/)A^2' 
R_0(ij) = R_i + R_j - δ; with δ = 0.01 [A]

350.000000      1.371000    # F_ C_3  (7)
350.000000      1.530000    # C_3 C_3 (8)
350.000000      1.420000    # C_3 O_3 (9)
""";

dct_bond_type = {"1-1":1,
                 "1-2":2,"2-1":2,
                 "2-3":3,"3-2":3,
                 "2-4":4,"4-2":4,
                 "2-5":5,"5-2":5,
                 "5-4":6,"4-5":6,
                 "3-1":7,"1-3":7,
                 "2-2":8,
                 "1-4":9
                }

In [23]:
# The ones not shown in the reference paper are kept as they are DREIDING model
MODEL_bonds["bond type"] = MODEL_bonds["X-X"].map(dct_bond_type).astype(int)

In [24]:
MODEL_angles = angles[["angle id","atom1","atom2","atom3"]].copy()
MODEL_angles["X-X-X"] = (
    MODEL_angles["atom1"].map(dct_types).astype(str) + "-" +
    MODEL_angles["atom2"].map(dct_types).astype(str) + "-" +
    MODEL_angles["atom3"].map(dct_types).astype(str) 
)

In [25]:
MODEL_angles["X-X-X"].unique()

array(['3-2-3', '3-2-2', '3-2-4', '2-2-4', '3-2-5', '2-2-5', '2-4-2',
       '3-1-3', '3-1-1', '1-1-1', '2-2-2', '2-5-4', '4-5-4', '3-1-4',
       '1-1-4', '1-4-2'], dtype=object)

In [26]:
atomX = MODEL_angles[MODEL_angles["X-X-X"].isin(['3-1-1'])]["atom3"].values
structure[structure["atom id"].isin(O_2742)]

,data,atom id,molecule id,atom type,MATSQ charge,x,y,z,dreiding elemts,charge
752,"[753, 1, 3, -0.24706, 99.776, 35.661, 33.163, ...",753,1,3,-0.24706,99.776,35.661,33.163,O_3,-0.2742
753,"[754, 1, 3, -0.24706, 57.296, 18.907, 22.364, ...",754,1,3,-0.24706,57.296,18.907,22.364,O_3,-0.2742
754,"[755, 1, 3, -0.24706, 36.096, 22.364, 36.069, ...",755,1,3,-0.24706,36.096,22.364,36.069,O_3,-0.2742
755,"[756, 1, 3, -0.24706, 78.576, 32.204, 19.458, ...",756,1,3,-0.24706,78.576,32.204,19.458,O_3,-0.2742
772,"[773, 1, 3, -0.24706, 72.407, 40.063, 21.166, ...",773,1,3,-0.24706,72.407,40.063,21.166,O_3,-0.2742
773,"[774, 1, 3, -0.24706, 93.607, 33.953, 41.022, ...",774,1,3,-0.24706,93.607,33.953,41.022,O_3,-0.2742
774,"[775, 1, 3, -0.24706, 29.927, 14.505, 34.361, ...",775,1,3,-0.24706,29.927,14.505,34.361,O_3,-0.2742
775,"[776, 1, 3, -0.24706, 51.126, 20.615, 14.505, ...",776,1,3,-0.24706,51.126,20.615,14.505,O_3,-0.2742
792,"[793, 1, 3, -0.24706, 36.662, 19.834, 20.381, ...",793,1,3,-0.24706,36.662,19.834,20.381,O_3,-0.2742
793,"[794, 1, 3, -0.24692, 15.463, 20.381, 35.142, ...",794,1,3,-0.24692,15.463,20.381,35.142,O_3,-0.2742


In [27]:
"""
The DREIDING FF has the following parameters:
50.000000    109.471000    # X C_3 X (happens to be the same as C-C-O(4))
50.000000    104.510000    # X O_3 X (not there, so I am adding it.)
""";

dct_angle_type = {"1-1-1":1,
                  "1-1-3":2, "3-1-1":2,
                  "3-1-3":3,
                  "2-2-4":4, "3-2-3":4, "3-2-2":4, 
                  "2-2-2":4, "3-1-4":4, "1-1-4":4,
                  "2-4-2":5,
                  "4-2-3":6, "3-2-4":6,
                  "3-2-5":7, "5-2-3":7,
                  "2-2-5":8, "5-2-2":8,
                  "2-5-4":9,
                  "4-5-4":10,
                  "1-4-2":11
                }

In [28]:
MODEL_angles["angle type"] = MODEL_angles["X-X-X"].map(dct_angle_type).astype(int)

In [29]:
sorted(MODEL_angles["angle type"].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [30]:
MODEL_dihedrals = dihedrals[["dihedral id","atom1","atom2","atom3","atom4"]].copy()

In [31]:
MODEL_dihedrals["X-X-X-X"] = (
    MODEL_dihedrals["atom1"].map(dct_types).astype(str) + "-" +
    MODEL_dihedrals["atom2"].map(dct_types).astype(str) + "-" +
    MODEL_dihedrals["atom3"].map(dct_types).astype(str) + "-" +
    MODEL_dihedrals["atom4"].map(dct_types).astype(str) 
)

### DIHEDRAL

DREIDING Force Field:

$$
    E_{\text{dihedral}} = \frac{1}{2} V_{J,K} \left[ 1 - \cos \left( n_{J,K}(\varphi - d)\right) \right]
$$

Notice:

$$
    E_{\text{dihedral}} = \frac{1}{2} V_{J,K} \left[ 1 + \cos \left( n_{J,K}\varphi\right) \right],\:\: d=180°
$$

MODEL's Dihedral:

$$
    E_{\text{dihedral}} = \frac{1}{2} V_{J,K} \left[ 1 - \cos \left( n_{J,K}\varphi \right) \right]
$$

LAMMPS Harmonic Dihedral:

$$
    E_{\text{dihedral}} = V_{J,K} \left[ 1 + d\cos \left( n_{J,K}\varphi\right) \right]
$$



Which would give the same value as the model with $V^{charm}_{I,J} = \frac{1}{2}V^{model}_{I,J}$

### TAKE NOTE ON HOW THE IONOMER PARAMETRIZATION CAN IMPROVE FOR LATER

In [32]:
"""
The DREIDING Force Field states:
A dihedral single bond involving two sp3 atoms (J,K = X_3) has
    V = 2.0 kcal/mol; n_JK=3; φ_0=180°( or 60°)
So,the dihedral parameters:
    1.00000    1   3     # X-C_3-C_3-X = X-C_3-O_3-X 
""";

dct_dieh_type = {"1-1-1-3":1, "1-1-1-4":1, "3-1-1-4":1, "4-1-1-1":1, "4-1-1-3":1, # "x-1-1-x":1
                 "1-1-1-1":2,
                 "3-1-1-1":3,
                 "3-1-1-3":4,
                 "1-1-4-2":5, "3-1-4-2":5, # "x-1-4-x": 
                 "x-1-5-x":6, # not recorded by MATSQ | I wonder how bad this is for the simulation.
}

## NOTICE `.fillna(7)`

In [33]:
MODEL_dihedrals["dihedral type"] = MODEL_dihedrals["X-X-X-X"].map(dct_dieh_type).fillna(7).astype(int)

In [35]:
MODEL_dihedrals["X-X-X-X"].unique()

array(['3-2-2-3', '3-2-2-5', '4-2-2-3', '4-2-2-5', '3-2-4-2', '2-2-4-2',
       '3-2-5-4', '2-2-5-4', '3-1-1-3', '3-1-1-1', '1-1-1-3', '1-1-1-1',
       '3-1-1-4', '1-1-1-4', '2-2-2-3', '4-1-1-3', '4-1-1-1', '3-1-4-2',
       '1-1-4-2', '3-2-2-2', '3-2-2-4', '4-2-2-2', '4-2-2-4', '3-2-4-1',
       '2-2-4-1'], dtype=object)

In [37]:
sorted(MODEL_dihedrals["dihedral type"].unique())

[1, 2, 3, 4, 5, 7]

## WRITE .mol FILE

In [40]:
def write_mol_file():
    f = open("IONOMER_12reps_MODIFIED_DREIDING_.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 30.09.24\n")
    f.write(f"\n{len(MODEL_structure)} atoms\n{len(MODEL_bonds)} bonds\n"+
            f"{len(MODEL_angles)} angles\n{len(MODEL_dihedrals)} dihedrals")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(MODEL_structure)):
        f.write(f"{MODEL_structure['atom id'][i]}\t{MODEL_structure['x'][i]}"+
                f"\t{MODEL_structure['y'][i]}\t{MODEL_structure['z'][i]}\n")
    f.write("\nTypes\n\n")
    for i in range(len(MODEL_structure)):
        f.write(f"{MODEL_structure['atom id'][i]}\t{MODEL_structure['type'][i]}\n")
    f.write("\nCharges\n\n")
    for i in range(len(MODEL_structure)):
        f.write(f"{MODEL_structure['atom id'][i]}\t{MODEL_structure['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(MODEL_bonds)):
        f.write(f"{MODEL_bonds['bond id'][i]}\t{MODEL_bonds['bond type'][i]}\t"+
                f"{MODEL_bonds['atom1'][i]}\t{MODEL_bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(MODEL_angles)):
        f.write(f"{MODEL_angles['angle id'][i]}\t{MODEL_angles['angle type'][i]}\t"+
                f"{MODEL_angles['atom1'][i]}\t{MODEL_angles['atom2'][i]}\t{MODEL_angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(MODEL_dihedrals)):
        f.write(f"{MODEL_dihedrals['dihedral id'][i]}\t{MODEL_dihedrals['dihedral type'][i]}"+
                f"\t{MODEL_dihedrals['atom1'][i]}\t{MODEL_dihedrals['atom2'][i]}\t"+
                f"{MODEL_dihedrals['atom3'][i]}\t{MODEL_dihedrals['atom4'][i]}\n")

In [41]:
write_mol_file()
! code IONOMER_12reps_MODIFIED_DREIDING_.mol

In [42]:
! cp IONOMER_12reps_MODIFIED_DREIDING_.mol /home/kchinas/Documents/PhD/LAMMPS/CL/3rdMODEL/Ionomer